# Lesson Introduction

Think of managing a team where each member has a unique skill — one searches the web, another reads files, and you, as the manager, assign tasks. In **AI**, you can build a similar setup: agents with specific abilities, and an _orchestrator_ agent that delegates tasks.

In this lesson, you'll learn how to use an agent as a tool for another agent. This lets you combine the strengths of multiple agents, making your AI solutions modular and powerful. By the end, you'll know how to wrap agents as tools, orchestrate their collaboration, and run complex workflows with just a few lines of code.

## Agents as Tools: The Concept and Wrapping Agents

The main idea is using an agent as a tool. In real life, you might ask a colleague to handle a task because they have the right expertise. In software, you can create agents that are experts in certain domains — like searching the web or reading files — and use them as tools for a higher-level agent.

### Why do this?

- **Modularity:** Each agent focuses on one job, making code easier to manage.
- **Reusability:** Specialized agents can be reused in different projects.
- **Separation of Concerns:** Agents can be developed and tested independently.

For example, to answer questions that need both web search and file reading, you can create two specialized agents and an orchestrator agent that knows when to use each.

To use an agent as a tool, you "wrap" it so another agent can call it like any other tool. The `Agents SDK` provides `.as_tool()` for this.

Here’s a basic example. Suppose you have a web search agent:

```python
from agents import Agent, WebSearchTool

web_search_agent = Agent(
    name="Web Search Agent",
    tools=[WebSearchTool()],
    instructions="You can use the web search tool to find information on the web."
)
```

To make this agent available as a tool:

```python
web_search_tool = web_search_agent.as_tool(
    tool_name="web_search",
    tool_description="A tool to search the web for information."
)
```

Now, `web_search_tool` can be added to another agent’s tools. The `tool_name` and `tool_description` help the orchestrator agent know what this tool does. You can do the same with any agent, including custom ones. For example, a filesystem agent that reads files can also be wrapped as a tool.

# Building an Orchestrator Agent

An _orchestrator agent_ is like a project manager. It doesn’t do the specialized work itself, but knows which agent to call for each task. You build it by adding other agents (wrapped as tools) to its `tools` list.

Example:
``` python
from agents import Agent

orchestrator = Agent(
    name="Orchestrator",
    tools=[web_search_tool, file_read_tool],  # Agents wrapped as tools
    instructions=(
        "You are an orchestrator agent. Use the web search tool to find information on the web "
        "and the file read tool to read the content of a file."
    )
)
```

The orchestrator’s instructions should explain when and how to use each tool. This helps the agent make the right decisions.

A real-life analogy: In a support center, the main agent (orchestrator) receives a question and forwards it to the right department (specialized agent). The orchestrator agent works the same way, routing tasks to the right specialist.

# Running the Orchestrator Agent

Once your orchestrator agent is set up, you interact with it like any other agent. The orchestrator decides which tool (specialized agent) to use for each request.

Example:
```python
from agents import Runner

# Ask the orchestrator to search the web
result = Runner.run_sync(orchestrator, "What is the weather in Tokyo?")
print(result.final_output)  # "The current weather in Tokyo is 22°C and sunny." (example output)

# Ask the orchestrator to read a file
result = Runner.run_sync(orchestrator, "What is the content of the file /usercode/FILESYSTEM/data.txt")
print(result.final_output)  # "Hello there"
```

## Lesson Summary and Practice Introduction

You’ve learned how to use agents as tools for other agents: how to wrap agents with `.as_tool()`, build an orchestrator agent that delegates tasks, and run complex workflows with clear, maintainable code. This modular approach mirrors real teamwork and makes your AI solutions flexible and scalable.

Now, it’s your turn to practice. In the next section, you’ll build and run your own orchestrator agent, combining specialized agents to solve real-world problems. This hands-on work will help you solidify your understanding and prepare for more advanced agent orchestration.

In [ ]:
# exercise 1

from agents import Agent, Runner, WebSearchTool, RunContextWrapper, FunctionTool
from pydantic import BaseModel
from typing import Any

def read_file(data):
    """Custom tool function that can be called by the agent."""
    try:
        with open(data, "r") as file:
            return file.read()
    except Exception as e:
        return f"Error reading file: {e}"

class FunctionArgs(BaseModel):
    file_path: str


async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return read_file(parsed.file_path)

file_read_tool = FunctionTool(
    name="file_read_tool",
    description="A tool to read the content of a file.",
    params_json_schema={
        "type": "object",
        "properties": {
            "file_path": {"type": "string"},
        },
        "required": ["file_path"],
        "additionalProperties": False
    },
    on_invoke_tool=run_function
)

agent1 = Agent(
    name="Web Search Agent",
    tools=[WebSearchTool()],
    instructions=(
        "You are a web search agent. You can use the web search tool to find information on the web."
    )
)

agent2 = Agent(
    name="Filesystem Agent",
    tools=[file_read_tool],
    instructions=(
        "You are a filesystem agent. You can use the file read tool to read the content of a file."
    )
)

orchestrator = Agent(
    name="Orchestrator",
    tools=[
        agent1.as_tool(
            tool_name="web_search",
            tool_description="A tool to search the web for information."
        ),
        agent2.as_tool(
            tool_name="file_read",
            tool_description="A tool to read the content of a file."
        )
    ],
    instructions=(
        # TODO: Enhance the instructions to let the orchestrator provide learning goals based on the file content at /usercode/FILESYSTEM/data.txt
        "You are an orchestrator agent. You can use the web search tool to find information on the web and the file read tool to read the content of a file."
    )
)


def main():
    result = Runner.run_sync(orchestrator, "What is the content of the file /usercode/FILESYSTEM/other.txt")
    print(result.final_output)

    # TODO: Add a call to the orchestrator to provide learning goals without mentioning the file name explicitely

if __name__ == "__main__":
    main()

In [ ]:
# exercise 1 - Francis solution

from agents import Agent, Runner, WebSearchTool, RunContextWrapper, FunctionTool
from pydantic import BaseModel
from typing import Any

def read_file(data):
    """Custom tool function that can be called by the agent."""
    try:
        with open(data, "r") as file:
            return file.read()
    except Exception as e:
        return f"Error reading file: {e}"

class FunctionArgs(BaseModel):
    file_path: str


async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return read_file(parsed.file_path)

file_read_tool = FunctionTool(
    name="file_read_tool",
    description="A tool to read the content of a file.",
    params_json_schema={
        "type": "object",
        "properties": {
            "file_path": {"type": "string"},
        },
        "required": ["file_path"],
        "additionalProperties": False
    },
    on_invoke_tool=run_function
)

agent1 = Agent(
    name="Web Search Agent",
    tools=[WebSearchTool()],
    instructions=(
        "You are a web search agent. You can use the web search tool to find information on the web."
    )
)

agent2 = Agent(
    name="Filesystem Agent",
    tools=[file_read_tool],
    instructions=(
        "You are a filesystem agent. You can use the file read tool to read the content of a file."
    )
)

orchestrator = Agent(
    name="Orchestrator",
    tools=[
        agent1.as_tool(
            tool_name="web_search",
            tool_description="A tool to search the web for information."
        ),
        agent2.as_tool(
            tool_name="file_read",
            tool_description="A tool to read the content of a file."
        )
    ],
    instructions=(
        # TODO: Enhance the instructions to let the orchestrator provide learning goals based on the file content at /usercode/FILESYSTEM/data.txt
        "You are an orchestrator agent. You can use the web search tool to find information on the web and the file read tool to read the content of a file. You can also provide learning goals based on the content of the file that you read at /usercode/FILESYSTEM/data.txt."
    )
)


def main():
    result = Runner.run_sync(orchestrator, "What is the content of the file /usercode/FILESYSTEM/other.txt")
    print(result.final_output)

    # TODO: Add a call to the orchestrator to provide learning goals without mentioning the file name explicitely
    result_learning = Runner.run_sync(orchestrator, "Provide learning goals")
    print(result_learning.final_output)

if __name__ == "__main__":
    main()